In [ ]:
!pip install tweepy

In [ ]:
!pip install textblob

# Ignore this section if you are doing locally

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
# open the file in the write mode
f = open("./drive/My Drive/new.csv", 'w')

# Start From Here

Library Imported

In [ ]:
import tweepy
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
consumer_key = "sbAnI5uPhos56fux65hxXwOE2"
consumer_secret = "oy3C8JLvk7vMihtIzaXdAAd4cK2k28lBDhBcGQlq882Uuj2X4O"
access_token = "20185148-y7BudlZah0DM05tnqKLNmPzX4lWNoL5wFVGe9Zlaz"
access_token_secret = "RVabWxPeqSmjwxGDxoanQnKUAleXaGclNVA92F5HTESe8"

Authentication using Twitter developer key and token

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

Sample tweet

In [ ]:
public_tweets = api.home_timeline()
# foreach through all tweets pulled
for tweet in public_tweets:
   # printing the text stored inside the tweet object
   print (tweet.text)

The Stanfield Conversations: Talking Democracy series at #DalhousieU aims to tackle subjects of national and intern… https://t.co/xdqcFK1lnW
RT @nytimes: With phone lines down and roadways disrupted, news organizations and emergency officials scrambled to gain access to parts of…
📧 Are you subscribed to our NASA Explore newsletter? This week, get info on a cargo delivery to the @Space_Station,… https://t.co/DmFCrJYghi
⛅ | Conditions at 6:04 pm: Mainly Sunny, 27.6°C. https://t.co/IdhYC3M2VB
RT @tianran: .⁦@RoryStewartUK⁩ notes Britain has cut 76%  of its aid to Afghanistan in the past year will end up dealing with an Afghan ref…
RT @DavidKlion: As with Vietnam, a lot of this is actually about refusing to accept that America lost a war, and that lives and treasure an…
👇🏽👇🏽👇🏽👇🏽👇🏽 https://t.co/jL9nnkpNBX
Let's write a compiler, part 1: Introduction, selecting a language, and planning https://t.co/juWPm4eA3A
Robotics roundup: Fields, food, fulfillment and Cyberdog https://t.co/dXCsalMGcS by

Fetching tweets in which #biden mentioned

In [ ]:
results = tweepy.Cursor(api.search, q="#biden", lang="en").items(1000000)

Applying Sentimental Analysis

In [ ]:
def clean_tweet(tweet):
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
def get_tweet_sentiment(tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

This function find state, country and **lang** & **lat** from Json

In [ ]:
def city_state_country(coord):
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    state = address.get('state', '')
    country = address.get('country', '')
    return  state, country

This API fetch location based on 

In [ ]:
import requests
import urllib.parse

address = 'Iowa, USA'
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

response = requests.get(url).json()

print(response[0]['display_name'])

Iowa, United States


Find Color name from Hax color code

In [ ]:
import re
re_color = re.compile('#([0-9a-f]{2})([0-9a-f]{2})([0-9a-f]{2})')
from math import sqrt

def color_to_rgb(hex):
     hex = hex.lstrip('#')
     hlen = len(hex)
     return tuple(int(hex[i:i+hlen/3], 16) for i in range(0, hlen, hlen/3))

def similarity(color1, color2):
    """Computes the pearson correlation coefficient for two colors. The result
    will be between 1.0 (very similar) and -1.0 (no similarity)."""
    c1 = color_to_rgb(color1)
    c2 = color_to_rgb(color2)

    s1 = sum(c1)
    s2 = sum(c2)
    sp1 = sum(map(lambda c: pow(c, 2), c1))
    sp2 = sum(map(lambda c: pow(c, 2), c2))
    sp = sum(map(lambda x: x[0] * x[1], zip(c1, c2)))

    try:
      computed = (sp - (s1 * s2 / 3.0)) / sqrt((sp1 - pow(s1, 2) / 3.0) * (sp2 - pow(s2, 2) / 3.0))
    except:
      computed = 0 
    return computed

color_names = {
    '#000000': 'black',
    '#ffffff': 'white',
    '#808080': 'dark gray',
    '#b0b0b0': 'light gray',
    '#ff0000': 'red',
    '#800000': 'dark red',
    '#00ff00': 'green',
    '#008000': 'darkgreen',
    '#0000ff': 'blue',
    '#000080': 'dark blue',
    '#ffff00': 'yellow',
    '#808000': 'olive',
    '#00ffff': 'cyan',
    '#ff00ff': 'magenta',
    '#800080': 'purple'
    }

def find_name(color):
    sim = [(similarity(color, c), name) for c, name in color_names.items()]
    return max(sim, key=lambda x: x[0])[1]


This Code only generate 5000 row in CSV file. Change **if** condition based on your requirement

In [ ]:
x=1
with open("./drive/My Drive/new.csv", 'w') as f:
  writer = csv.writer(f)

  header = ['ID', 'text', 'Sentiment','Description', 'source','profile_background_color','created_at','followers_count','lat','lon','display_name','verified','favourites_count','retweet_count','statuses_count']
  writer.writerow(header)
  for tweet in results:
    l=[]  
    address = tweet.user.location
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
    response = requests.get(url).json()
    if(x==5000):
      break
    if(len(response)!=0):
      l.append(x)
      x=x+1
      l.append(tweet.text)
      l.append(tweet.user.description)
      l.append(get_tweet_sentiment(tweet.text))
      l.append(tweet.source)
      l.append(tweet.user.profile_background_color)
      l.append(tweet.user.created_at)
      l.append(tweet.user.followers_count)
      l.append(response[0]["lat"])
      l.append(response[0]["lon"])
      l.append(city_state_country((response[0]["lat"],response[0]["lon"])))
      l.append(tweet.user.verified)
      l.append(tweet.user.favourites_count)
      l.append(tweet.retweet_count)
      l.append(tweet.user.statuses_count)
      writer.writerow(l)
  f.close()

#done with the code- chcek CSV file

## This section Allow you to find tweet based on Text and you can apply your Logic to create Dataset

In [ ]:
results = api.search(q="india", lang="en")

# foreach through all tweets pulled
for tweet in results:
   # printing the text stored inside the tweet object
   print (tweet.user.screen_name,"Tweeted:",tweet.text,"\n")

noteigenvector Tweeted: @rranjan_jms @globaltimesnews neche add kr bsdk "Kashmir is not India" 

missp000 Tweeted: RT @SkyCricket: INDIA 8️⃣ DOWN...

Two wickets in the over for Boult as, after removing Pant, he has Ashwin caught on the drive at slip #IN… 

Virendr70147086 Tweeted: RT @AryanGrover11: @KUNALGU12153401 @TIETofficial @ugc_india Exactly, We even called Wheebox Team, They are saying that it's not in their h… 

Althafps619 Tweeted: RT @benmammutty: After Hindu Banks, keenly waiting to see Hindu Petrol Bunks where no petrol/diesel from Muslim countries, but only from In… 

Shweta_AOL Tweeted: RT @OfficeOfGurudev: Modern Medicine and Yoga :

American College of Physicians India Chapter in conversation with Gurudev @SriSri Ravi Sha… 

Desi36348029 Tweeted: RT @sanjiv_bhasin: My original call on South India Bank @ 8.50, watch the video for target 25 &amp; press like😎
Sanjiv Bhasin's Target of South… 

Kakashi38933508 Tweeted: @AatmanKamdar Easy win for India. Nz all out for 79. 